# Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [ ]:
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
training_labels = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')

In [ ]:
training_labels

# Demonstration of the Data

In [ ]:
my_tensor = np.load('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy', allow_pickle=True)
my_tensor = np.load('../input/g2net-gravitational-wave-detection/test/0/0/0/00024887b5.npy', allow_pickle=True)

max_of_rows = my_tensor.max(axis=1)
my_tensor = my_tensor / max_of_rows[:, np.newaxis]

In [ ]:
label = training_labels.id[0]
target = training_labels.target[0]

In [ ]:
plt.figure(num=0, figsize= (9,9))
for i in range(3):
    plt.subplot(3, 1, i+1)
    plt.title(f'target: {target}, label: {label}')
    plt.plot(my_tensor[i])

# What the neural net sees after GlobalAveragePooling2D()

In [ ]:
my_tensor = np.load('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy', allow_pickle=True)
my_tensor = np.load('../input/g2net-gravitational-wave-detection/test/0/0/0/0003259f74.npy', allow_pickle=True)

print(my_tensor.shape)

max_of_rows = my_tensor.max(axis=1)
my_tensor = my_tensor / max_of_rows[:, np.newaxis]

d1 = my_tensor[0]
d2 = my_tensor[1]
d3 = my_tensor[2]

d1 = d1.reshape(64, 64, 1)
d2 = d2.reshape(64, 64, 1)
d3 = d3.reshape(64, 64, 1)

d = np.concatenate((d1,d2,d3), axis=2)

plt.imshow(d)

D = layers.GlobalAveragePooling2D()(d.reshape(1, 64, 64, 3))


plt.imshow(D)

# Loading the Data

In [ ]:
flag = 0
tensor_list = []
for dirname, _, filenames in os.walk('../input/g2net-gravitational-wave-detection/train'):
    for filename in filenames:
        tensor_path = os.path.join(dirname, filename)
        my_tensor = np.load(tensor_path, allow_pickle=True)
        max_of_rows = my_tensor.max(axis=1)
        my_tensor = my_tensor / max_of_rows[:, np.newaxis]
        
        d1 = my_tensor[0]
        d2 = my_tensor[1]
        d3 = my_tensor[2]

        d1 = d1.reshape(64, 64, 1)
        d2 = d2.reshape(64, 64, 1)
        d3 = d3.reshape(64, 64, 1)

        d = np.concatenate((d1,d2,d3), axis=2)
        
        tensor_list.append(d)
        flag += 1
    if flag > 1000:
        break
        

In [ ]:
X_train = np.array(tensor_list)
y_train = training_labels.target.to_numpy()

In [ ]:
X_train.shape

# Exhibition of the Data

In [ ]:
plt.imshow(X_train[0])

In [ ]:
y_train = y_train[:len(X_train)]

In [ ]:
X_train.shape

In [ ]:
X_train.shape

# Building the Model

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

res_model = ResNet50(input_tensor=inputs, weights='imagenet')

In [ ]:
len(res_model.layers)

In [ ]:
for layer in res_model.layers[:161]:
    layer.trainable = False

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
scheduler_callback =  tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.1, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs'),
    early_stopping,
    scheduler_callback
]

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

inputs = keras.Input(shape=(64, 64, 3))
pre = preprocessing.Resizing(int(224), int(224), interpolation='bilinear')(inputs)
x = res_model(pre)
x = layers.Flatten()(x)
x = layers.Dense(100, activation="sigmoid")(x)
outputs = layers.Dense(2, activation="softmax")(x)

In [ ]:
from tensorflow.keras import layers

inputs = keras.Input(shape=(64, 64, 3))
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
# x = layers.MaxPooling2D(pool_size=(2, 2))(x)
# x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=(3, 3))(x)
# x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 2
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
y_train[:10]

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
y_train = y_train[:len(X_train)]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.2, random_state=1)

In [ ]:
X_train[0].shape

# Training the Model

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_valid, y_valid),
                   callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(num=0)
plt.title('loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()

plt.figure(num=1)
plt.title('accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()

In [ ]:
loss, acc = model.evaluate(X_test, y_test)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

# Predictions

In [ ]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)    

In [ ]:
predictions

In [ ]:
X_train.shape